<a href="https://colab.research.google.com/github/klnsuman/LangChain/blob/main/Prompts_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
drive.mount('/content/drive')
import os
import sys
sys.path.append("/content/drive/MyDrive/colab_env/lib/python3.10/site-packages")
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
OpenAI API Key:··········


In [18]:
#!pip install virtualenv

In [23]:
#!virtualenv /content/drive/MyDrive/colab_env
#!source /content/drive/MyDrive/colab_env/bin/activate; pip install langchain
#!source /content/drive/MyDrive/colab_env/bin/activate; pip install chromadb openai tiktoken faiss-cpu

# ***Prompts***

In [12]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")
prompt = """
Today is Monday tomorrow is saturday

what is wrong with statement?
"""

llm(prompt)

'\nThe statement is incorrect because Monday and Saturday are not consecutive days of the week.'

In [8]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")
prompt = """
What is capital of India
"""
print(llm(prompt))

prompt = """
How much is 2 greater than 1
"""
print(llm(prompt))


The capital of India is New Delhi.



2 is 1 greater than 1, so the answer is 1.


In [9]:
prompt = """
How long does Tortoise Live
"""

llm(prompt)

'\nTortoises can live for up to 150 years or more, depending on the species. Some species of tortoise, such as the giant Galapagos tortoise, can live for over 200 years.'

# ***Prompt Templates***

In [5]:
from langchain.prompts import PromptTemplate
prompt_template = " What is the capital city of {country}"
llm = OpenAI(model_name="text-davinci-003")

prompt = PromptTemplate(input_variables = ["country"],template=prompt_template)

final_prompt = prompt.format(country='India')
print(final_prompt)
print(llm(final_prompt))

 What is the capital city of India
?

The capital city of India is New Delhi.


# ***Example Selectors***

In [29]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptWithTemplates,FewShotPromptTemplate,PromptTemplate
from langchain.llms import OpenAI

In [13]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

# Examples of locations that nouns are found
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

In [25]:
# SemanticSimilarityExampleSelector will select examples that are similar to your input by semantic meaning

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,

    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),

    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,

    # This is the number of examples to produce.
    k=2
)

In [30]:
similar_prompt = FewShotPromptTemplate(
    # The object that will help select examples
    example_selector=example_selector,

    # Your prompt
    example_prompt=example_prompt,

    # Customizations that will be added to the top and bottom of your prompt
    prefix="Give the location an item is usually found in",
    suffix="Input: {noun}\nOutput:",

    # What inputs your prompt will receive
    input_variables=["noun"],
)


In [31]:
# Select a noun!
my_noun = "student"

print(similar_prompt.format(noun=my_noun))

Give the location an item is usually found in

Example Input: driver
Example Output: car

Example Input: pilot
Example Output: plane

Input: student
Output:


In [32]:
llm(similar_prompt.format(noun=my_noun))

' classroom'

# **OUTPUT Parser**

In [33]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

In [35]:
llm = OpenAI(model_name="text-davinci-003")

In [36]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [37]:
# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```


In [41]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="yelcom to Hyd!")

print(promptValue)


You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```

% USER INPUT:
yelcom to Hyd!

YOUR RESPONSE:



In [42]:
llm_output = llm(promptValue)
llm_output

'```json\n{\n\t"bad_string": "yelcom to Hyd!",\n\t"good_string": "Welcome to Hyde!"\n}\n```'

In [43]:
output_parser.parse(llm_output)

{'bad_string': 'yelcom to Hyd!', 'good_string': 'Welcome to Hyde!'}